In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install transformers datasets torch evaluate accelerate


In [2]:
import torch
from datasets import load_dataset
from transformers import (
    AutoTokenizer, AutoModelForSeq2SeqLM, 
    Trainer, TrainingArguments, DataCollatorForSeq2Seq
)

# Initial model

In [2]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

# Load the BERT-based model
model_name = "t5-small"  # You can also try "facebook/bart-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

In [ ]:
from datasets import load_dataset

dataset = load_dataset("esnli")  # Explainable NLI dataset
train_data = dataset["train"]


In [ ]:
def preprocess_data(examples):
    inputs = ["Premise: " + p + " Hypothesis: " + h for p, h in zip(examples["premise"], examples["hypothesis"])]
    outputs = examples["explanation_1"]
    model_inputs = tokenizer(inputs, truncation=True, padding="max_length", max_length=512)
    labels = tokenizer(outputs, truncation=True, padding="max_length", max_length=128)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_dataset = dataset.map(preprocess_data, batched=True)

In [ ]:
small_train_dataset = tokenized_dataset["train"].shuffle(seed=42).select(range(5000))  # Use only 5000 samples
small_eval_dataset = tokenized_dataset["validation"].shuffle(seed=42).select(range(1000))  # Use only 1000 samples

In [3]:
import os
os.environ["WANDB_DISABLED"] = "true"

In [ ]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    per_device_train_batch_size=4,  # Keep small for Kaggle GPU
    per_device_eval_batch_size=4,
    num_train_epochs=3,  # Adjust for better training
    weight_decay=0.01,
    save_total_limit=2,
    load_best_model_at_end=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
)

trainer.train()

In [ ]:
def generate_explanation(premise, hypothesis):
    input_text = f"Premise: {premise} Hypothesis: {hypothesis}"
    inputs = tokenizer(input_text, return_tensors="pt").to("cuda")  # Move to GPU
    output = model.generate(**inputs, max_length=128)
    return tokenizer.decode(output[0], skip_special_tokens=True)

# Example
premise = "The weather is clear and sunny."
hypothesis = "It is not raining."
print(generate_explanation(premise, hypothesis))

In [ ]:
model.save_pretrained("./fine_tuned_bert_xai")
tokenizer.save_pretrained("./fine_tuned_bert_xai")

In [ ]:
def generate_explanation(premise, hypothesis):
    input_text = f"Explain why: {premise} Therefore, {hypothesis}"
    inputs = tokenizer(input_text, return_tensors="pt").to("cuda")
    output = model.generate(**inputs, max_length=128)
    return tokenizer.decode(output[0], skip_special_tokens=True)

# Example Test
premise = "The sky is cloudy."
hypothesis = "It might rain soon."
print(generate_explanation(premise, hypothesis))

# Improving results with a different dataset

In [4]:
dataset = load_dataset("cos_e", "v1.11")
print(dataset["train"].column_names)

README.md:   0%|          | 0.00/7.87k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/1.73M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/222k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/9741 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1221 [00:00<?, ? examples/s]

['id', 'question', 'choices', 'answer', 'abstractive_explanation', 'extractive_explanation']


In [5]:
def preprocess_data(examples):
    inputs = [f"Explain: {q} Choices: {c} Answer: {examples['answer'][i]}" 
              for i, (q, c) in enumerate(zip(examples["question"], examples["choices"]))]
    outputs = examples["abstractive_explanation"]  # Use correct column name
    
    model_inputs = tokenizer(inputs, truncation=True, padding="max_length", max_length=512)
    labels = tokenizer(outputs, truncation=True, padding="max_length", max_length=128)
    
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_dataset = dataset.map(preprocess_data, batched=True)

Map:   0%|          | 0/9741 [00:00<?, ? examples/s]

Map:   0%|          | 0/1221 [00:00<?, ? examples/s]

In [ ]:
training_args = TrainingArguments(
    output_dir="./results_cosE",
    per_device_train_batch_size=4,  
    per_device_eval_batch_size=4,  
    num_train_epochs=3,  
    fp16=True,  # Enable mixed precision for faster training
    save_total_limit=1,
    evaluation_strategy="epoch",
    logging_strategy="epoch"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
)

trainer.train()

# More epochs, with weight decay

In [ ]:
training_args = TrainingArguments(
    output_dir="./results_cosE2",
    per_device_train_batch_size=4,  
    per_device_eval_batch_size=4,  
    num_train_epochs=5,  
    fp16=True,  # Enable mixed precision for faster training
    save_total_limit=1,
    evaluation_strategy="epoch",
    logging_strategy="epoch",
    weight_decay=0.01,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
)

trainer.train()

In [ ]:
training_args = TrainingArguments(
    output_dir="./results_cosE3",
    per_device_train_batch_size=4,  
    per_device_eval_batch_size=4,  
    num_train_epochs=5,  
    fp16=True,  # Enable mixed precision for faster training
    save_total_limit=1,
    evaluation_strategy="epoch",
    logging_strategy="epoch",
    learning_rate=2e-5,
    weight_decay=0.01
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
)

trainer.train()

In [ ]:
def generate_explanation(question, choices, answer):
    input_text = f"Question: {question} Answer: {answer}. Explain why in a clear and detailed way."
    inputs = tokenizer(input_text, return_tensors="pt").to("cuda")

    output = model.generate(
        **inputs,
        max_length=128,  
        min_length=50,   # Forces detailed explanations
        do_sample=True,  
        temperature=0.7,  
        top_p=0.9,  
        repetition_penalty=1.5,  # Penalizes repeating phrases
    )

    return tokenizer.decode(output[0], skip_special_tokens=True)

question = "Why do we wear sunglasses?"
choices = "['To look stylish', 'To protect our eyes', 'To hide from people']"
answer = "To protect our eyes"

print(generate_explanation(question, choices, answer))

In [12]:
training_args = TrainingArguments(
    output_dir="./results_cosE_best",
    per_device_train_batch_size=4,  
    per_device_eval_batch_size=4,  
    num_train_epochs=5,  
    fp16=True,  # Enable mixed precision for faster training
    save_total_limit=1,
    evaluation_strategy="epoch",
    logging_strategy="epoch",
    learning_rate=2e-5,
    weight_decay=0.001
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
)

trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked t

Epoch,Training Loss,Validation Loss
1,0.647000,0.253465
2,0.242000,0.242248
3,0.229100,0.240974
4,0.224800,0.240255
5,0.222600,0.239898


/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked t

TrainOutput(global_step=6090, training_loss=0.31311624461206894, metrics={'train_runtime': 1737.0511, 'train_samples_per_second': 28.039, 'train_steps_per_second': 3.506, 'total_flos': 6591822440693760.0, 'train_loss': 0.31311624461206894, 'epoch': 5.0})

In [16]:
model_path = "./cose_model"  # Make sure this is the correct path
model = AutoModelForSeq2SeqLM.from_pretrained(model_path).to("cuda")
tokenizer = AutoTokenizer.from_pretrained(model_path)

In [22]:
def generate_explanation(question, choices, answer):
    input_text = f"Question: {question} Answer: {answer}."
    inputs = tokenizer(input_text, return_tensors="pt").to("cuda")
    inputs = {key: val.to("cuda") for key, val in inputs.items()}
    output = model.generate(
        **inputs,
        max_length=128,  
        min_length=10,   # Forces detailed explanations
        do_sample=True,  
        temperature=0.7,  
        top_p=0.9,  
        repetition_penalty=1.5,  # Penalizes repeating phrases
    )

    return tokenizer.decode(output[0], skip_special_tokens=True)

question = "Why did the AI flag this transaction as fraud? "
choices = "['Transaction amount was too high', 'Location was unusual', 'User behavior was suspicious']"
answer = "User behavior was suspicious"

print(generate_explanation(question, choices, answer))

user behavior was suspicious. the AI flag this transaction as fraud


In [15]:
model.save_pretrained("./cose_model")
tokenizer.save_pretrained("./cose_model")

('./cose_model/tokenizer_config.json',
 './cose_model/special_tokens_map.json',
 './cose_model/spiece.model',
 './cose_model/added_tokens.json',
 './cose_model/tokenizer.json')

In [6]:
training_args = TrainingArguments(
    output_dir="./results_cosE_longer",
    per_device_train_batch_size=8,  
    per_device_eval_batch_size=8,  
    num_train_epochs=15,  
    fp16=True,  # Enable mixed precision for faster training
    save_total_limit=1,
    evaluation_strategy="epoch",
    logging_strategy="epoch",
    learning_rate=2e-5,
    weight_decay=0.001
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
)

trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked t

Epoch,Training Loss,Validation Loss
1,0.911500,0.275559
2,0.255700,0.247064
3,0.234200,0.242684
4,0.226700,0.240452
5,0.222000,0.239260
6,0.219300,0.240031
7,0.216600,0.239584
8,0.214800,0.237638
9,0.213500,0.238217
10,0.211700,0.237921


/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked t

TrainOutput(global_step=9135, training_loss=0.2649263278604141, metrics={'train_runtime': 4484.4588, 'train_samples_per_second': 32.583, 'train_steps_per_second': 2.037, 'total_flos': 1.977546732208128e+16, 'train_loss': 0.2649263278604141, 'epoch': 15.0})

In [7]:
model.save_pretrained("./cose_model_v2")
tokenizer.save_pretrained("./cose_model_v2")

('./cose_model_v2/tokenizer_config.json',
 './cose_model_v2/special_tokens_map.json',
 './cose_model_v2/spiece.model',
 './cose_model_v2/added_tokens.json',
 './cose_model_v2/tokenizer.json')

In [3]:
model_path = "./cose_model_v2"  # Make sure this is the correct path
model = AutoModelForSeq2SeqLM.from_pretrained(model_path).to("cuda")
tokenizer = AutoTokenizer.from_pretrained(model_path)

In [8]:
def generate_explanation(question, choices, answer):
    input_text = f"Question: {question} Answer: {answer}."
    inputs = tokenizer(input_text, return_tensors="pt").to("cuda")
    inputs = {key: val.to("cuda") for key, val in inputs.items()}
    output = model.generate(
        **inputs,
        max_length=128,  
        min_length=10,   # Forces detailed explanations
        do_sample=True,  
        temperature=0.7,  
        top_p=0.9,  
        repetition_penalty=1.5,  # Penalizes repeating phrases
    )

    return tokenizer.decode(output[0], skip_special_tokens=True)

question = "Why did the AI flag this transaction as fraud? "
choices = "['Transaction amount was too high', 'Location was unusual', 'User behavior was suspicious']"
answer = "User behavior was suspicious"

print(generate_explanation(question, choices, answer))

the AI flag this transaction as fraud. User behavior was suspicious


# Cos E with facebook bart

In [38]:
model_name = "facebook/bart-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

In [25]:
training_args = TrainingArguments(
    output_dir="./results_bart",
    per_device_train_batch_size=4,  
    per_device_eval_batch_size=4,  
    num_train_epochs=5,  
    fp16=True,  # Enable mixed precision for faster training
    save_total_limit=1,
    evaluation_strategy="epoch",
    logging_strategy="epoch",
    learning_rate=2e-5,
    weight_decay=0.01
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
)

trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,0.409800,0.365140
2,0.331500,0.367349
3,0.311800,0.368198
4,0.298700,0.370502
5,0.290600,0.371291


/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:2817: UserWarning: Moving the following attributes in the config to the generation config: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tens

TrainOutput(global_step=6090, training_loss=0.3284676838390933, metrics={'train_runtime': 3315.6211, 'train_samples_per_second': 14.69, 'train_steps_per_second': 1.837, 'total_flos': 1.48486070992896e+16, 'train_loss': 0.3284676838390933, 'epoch': 5.0})

In [26]:
model.save_pretrained("./bart_model_v1")
tokenizer.save_pretrained("./bart_model_v1")

('./bart_model_v1/tokenizer_config.json',
 './bart_model_v1/special_tokens_map.json',
 './bart_model_v1/vocab.json',
 './bart_model_v1/merges.txt',
 './bart_model_v1/added_tokens.json',
 './bart_model_v1/tokenizer.json')

In [56]:
model_path = "./bart_model_v1"  # Make sure this is the correct path
model = AutoModelForSeq2SeqLM.from_pretrained(model_path).to("cuda")
tokenizer = AutoTokenizer.from_pretrained(model_path)

In [59]:
def generate_explanation(question, choices, answer):
    input_text = f"Question: {question} Answer: {answer}."
    inputs = tokenizer(input_text, return_tensors="pt").to("cuda")
    inputs = {key: val.to("cuda") for key, val in inputs.items()}
    output = model.generate(
        **inputs,
        max_length=128,  
        min_length=10,   # Forces detailed explanations
        do_sample=True,  
        temperature=0.7,  
        top_p=0.9,  
        repetition_penalty=1.5,  # Penalizes repeating phrases
    )

    return tokenizer.decode(output[0], skip_special_tokens=True)

question = "Why did the AI flag this transaction as fraud? "
choices = "['Transaction amount was too high', 'Location was unusual', 'User behavior was suspicious']"
answer = "User behavior was suspicious"

print(generate_explanation(question, choices, answer))

 Question: Why did the AI flag this transaction as fraud?  Answer: User behavior was suspicious.


In [39]:
training_args = TrainingArguments(
    output_dir="./results_bart_v2",
    per_device_train_batch_size=8,  
    per_device_eval_batch_size=8,  
    num_train_epochs=5,  
    fp16=True,  # Enable mixed precision for faster training
    save_total_limit=1,
    evaluation_strategy="epoch",
    logging_strategy="epoch",
    learning_rate=2e-5,
    weight_decay=0.001
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
)

trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,0.495700,0.364151
2,0.342500,0.366109
3,0.323800,0.368624
4,0.313500,0.370381
5,0.307200,0.369693


/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:2817: UserWarning: Moving the following attributes in the config to the generation config: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tens

TrainOutput(global_step=3045, training_loss=0.35653723104442475, metrics={'train_runtime': 2753.4343, 'train_samples_per_second': 17.689, 'train_steps_per_second': 1.106, 'total_flos': 1.48486070992896e+16, 'train_loss': 0.35653723104442475, 'epoch': 5.0})

In [40]:
model.save_pretrained("./bart_model_v2")
tokenizer.save_pretrained("./bart_model_v2")

('./bart_model_v2/tokenizer_config.json',
 './bart_model_v2/special_tokens_map.json',
 './bart_model_v2/vocab.json',
 './bart_model_v2/merges.txt',
 './bart_model_v2/added_tokens.json',
 './bart_model_v2/tokenizer.json')

In [52]:
model_path = "./bart_model_v2"  # Make sure this is the correct path
model = AutoModelForSeq2SeqLM.from_pretrained(model_path).to("cuda")
tokenizer = AutoTokenizer.from_pretrained(model_path)

In [55]:
def generate_explanation(question, choices, answer):
    input_text = f"Question: {question} Answer: {answer}."
    inputs = tokenizer(input_text, return_tensors="pt").to("cuda")
    inputs = {key: val.to("cuda") for key, val in inputs.items()}
    output = model.generate(
        **inputs,
        max_length=128,  
        min_length=10,   # Forces detailed explanations
        do_sample=True,  
        temperature=0.7,  
        top_p=0.9,  
        repetition_penalty=1.5,  # Penalizes repeating phrases
    )

    return tokenizer.decode(output[0], skip_special_tokens=True)

question = "Why did the AI flag this transaction as fraud? "
choices = "['Transaction amount was too high', 'Location was unusual', 'User behavior was suspicious']"
answer = "User behavior was suspicious"

print(generate_explanation(question, choices, answer))

Question: Why did the AI flag this transaction as fraud?  Answer: User behavior was suspicious.
